In [2]:
!pip install requests
!pip install bs4
!pip install lxml
!pip install pandas

In [1]:
# import libraries
import os
import pandas as pd
import lxml
import requests  
from bs4 import BeautifulSoup
from scraper import *

# scraping of the performance info

In [1]:
from scraper import *
from utils import *

list_url = ['https://www.vlr.gg/296735/t1-vs-bleed-champions-tour-2024-pacific-kickoff-opening-a', 'https://www.vlr.gg/296730/zeta-division-vs-global-esports-champions-tour-2024-pacific-kickoff-opening-a']

test = general_data_scraper(list_url)

In [1]:
from scraper import *

main("https://www.vlr.gg/event/matches/1924/champions-tour-2024-pacific-kickoff/?series_id=all")

Get all the matches...
Fetching General Data...
Fetching Performance Data...
Fetching Economy Data...
Fetching Picks and Bans Data...
Saving the data...
DataFrame saved as CSV file: champions-tour-2024-pacific-kickoff_data\general_data_champions-tour-2024-pacific-kickoff.csv
DataFrame saved as CSV file: champions-tour-2024-pacific-kickoff_data\performance_data_champions-tour-2024-pacific-kickoff.csv
DataFrame saved as CSV file: champions-tour-2024-pacific-kickoff_data\economy_data_champions-tour-2024-pacific-kickoff.csv
DataFrame saved as CSV file: champions-tour-2024-pacific-kickoff_data\pick_ban_data_champions-tour-2024-pacific-kickoff.csv


0

In [6]:
result.head()

,Stage,Series,Team Name,Bans,Picks,Decider
0,Group Stage,Decider (C),M8,"[Bind, Ascent]",[Breeze],[Sunset]
1,Group Stage,Decider (C),VIT,"[Icebox, Split]",[Lotus],[Sunset]
2,Play-Ins,Group B,GTZ,"[Lotus, Split, Haven]",[],[Pearl]
3,Play-Ins,Group B,MOUZ,"[Fracture, Ascent, Bind]",[],[Pearl]
